# Generating texts
In this notebook we will generate texts and store them in a CSV file. More precisely, we generate random PhD topics which could potentially be worked on at the UFZ Leipzig structured as described in its [organigram](https://www.ufz.de/export/data/global/98993_organisation-chart_27-10-2025.jpg).

In [1]:
import bia_bob
import random

First, we test the connection to the LLM server.

In [2]:
def prompt_ollama(message:str, model="gemma3:12b", temperature=0.2):
    """A prompt helper function that sends a message to ScaDS.AI LLM server at 
    ZIH TU Dresden and returns only the text response.
    """
    import os
    import openai
    
    # convert message in the right format if necessary
    if isinstance(message, str):
        message = [{"role": "user", "content": message}]
    
    # setup connection to the LLM
    client = openai.OpenAI(base_url="http://localhost:11434/v1",
                           api_key = "none"
            #base_url="https://chat-ai.academiccloud.de/v1",
            #api_key = os.environ.get('KISSKI_API_KEY')
            #base_url="https://llm.scads.ai/v1",
            #api_key=os.environ.get('SCADSAI_API_KEY')
                           # openai/gpt-oss-120b
    )
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature
    )
    
    # extract answer
    return response.choices[0].message.content

prompt_ollama("hello world")

"Hello there! 👋 \n\nIt's great to see you. Is there anything I can help you with today?\n"

Next, we define a couple of random first and lastnames. We also define a list of institutes.
These will hint the LLM to generate PhD topics that are relevant to a given institute.

In [3]:
import pandas as pd
import numpy as np
import random

# Define separate lists for first names and last names
first_names = ["Alex", "Jordan", "Morgan", "Taylor", "Avery",
               "Sam", "Casey", "Riley", "Jamie", "Dakota",
               "Bailey", "Cameron", "Sydney", "Robin", "Charlie",
               "Dana", "Kris", "Skyler", "Devon", "Reese"]

last_names = ["Reed", "Smith", "Lee", "Garcia", "Patel",
              "Chen", "O'Hara", "Jain", "Kumar", "Singh",
              "Liu", "Davis", "Clark", "Thomas", "Brooks",
              "Flores", "Rivera", "Adams", "Gonzalez", "Campbell"]

faculties = """Ecosystems of the Future / Community Ecology
Ecosystems of the Future / Biodiversity and People
Ecosystems of the Future / Ecology of Agroecosystems
Ecosystems of the Future / Soil System Science
Ecosystems of the Future / Computational Landscape Ecology
Ecosystems of the Future / Conservation Biology and Social-Ecological Systems
Ecosystems of the Future / Physiological Diversity
Ecosystems of the Future / Species Interaction Ecology
Water Resources and Environment / Aquatic Ecosystem Analysis
Water Resources and Environment / Catchment Hydrology
Water Resources and Environment / River Ecology
Water Resources and Environment / Hydrogeology
Water Resources and Environment / Lake Research
Chemicals in the Environment / Computational Biology & Chemistry
Chemicals in the Environment / Environmental Analytical Chemistry
Chemicals in the Environment / Exposure Science
Chemicals in the Environment / Molecular Toxicology
Chemicals in the Environment / Ecotoxicology
Chemicals in the Environment / Environmental Immunology
Chemicals in the Environment / Cell Toxicology
Sustainable Ecotechnologies / Applied Microbial Ecology
Sustainable Ecotechnologies / Solar Materials Biotechnology
Sustainable Ecotechnologies / Microbial Biotechnology
Sustainable Ecotechnologies / Molecular Environmental Biotechnology
Sustainable Ecotechnologies / Systemic Environmental Biotechnology
Sustainable Ecotechnologies / Technical Biogeochemistry
Smart Models / Monitoring / Compound Environmental Risks
Smart Models / Monitoring / Computational HydroSystems
Smart Models / Monitoring / Monitoring & Exploration Technologies
Smart Models / Monitoring / Ecological Modelling
Smart Models / Monitoring / Remote Sensing
Smart Models / Monitoring / Environmental Informatics
Environment and Society / System Analysis and Sustainability Assessment
Environment and Society / Economics
Environment and Society / Urban & Environmental Sociology
Environment and Society / Environmental Politics
Environment and Society / Environmental & Planning Law""".split("\n")
len(faculties), faculties[:10]

(37,
 ['Ecosystems of the Future / Community Ecology',
  'Ecosystems of the Future / Biodiversity and People',
  'Ecosystems of the Future / Ecology of Agroecosystems',
  'Ecosystems of the Future / Soil System Science',
  'Ecosystems of the Future / Computational Landscape Ecology',
  'Ecosystems of the Future / Conservation Biology and Social-Ecological Systems',
  'Ecosystems of the Future / Physiological Diversity',
  'Ecosystems of the Future / Species Interaction Ecology',
  'Water Resources and Environment / Aquatic Ecosystem Analysis',
  'Water Resources and Environment / Catchment Hydrology'])

In the following loop we select an institute name randomly and ask the LLM server to generate a PhD topic related to the chosen institute.

In [4]:
research_fields = faculties

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

profiles = []

#for field in faculties:
while len(profiles) < 250:    
    # Random name generation from first and last name
    name = f"{random.choice(first_names)} {random.choice(last_names)}"
    field = random.choice(research_fields)

    topic = prompt_ollama(f"""Generate an English PhD thesis topic for someone working at an Environmental Research Center in group '{field}'. Reply with only the title of the research project and nothing else.""", temperature=0.9)
    topic = topic.strip().replace("</s>", "").replace("\"", "")
    if ":" in topic and " " not in topic.split(":")[0]:
        topic = topic.split(":")[1]
    if "</thinking>" in topic:
        topic = topic.split("</thinking>")[1]

    print(len(topic), topic)
    
    profiles.append({
        "name": name,
        "research_field": field,
        "topic": topic
    })

120 Microplastic-Associated Persistent Organic Pollutants: Bioaccumulation, Trophic Transfer, and Ecological Risk Assessment
74 Microbial Community Resilience to Agricultural Runoff in Headwater Streams
108 Resilience and Relocation: Social-Ecological Pathways for Coastal Community Adaptation in a Changing Climate
114 Resilience and Adaptive Capacity: Integrating Soil Microbiome Dynamics and Crop Diversity in Future Agroecosystems
131 Predicting Persistent Organic Pollutant Bioaccumulation and Toxicity Using Integrated Machine Learning and Physiochemical Modeling.
140 The Intertwined Futures: Indigenous Ecological Knowledge, Climate Change Adaptation, and Biodiversity Conservation in a Fragmented Landscape
129 The Environmental Justice of Urban Heat Islands: A Mixed-Methods Analysis of Vulnerability, Adaptation, and Community Resilience.
95 AI-Driven Anomaly Detection in Environmental Sensor Networks: A Bayesian Deep Learning Approach
92 Resilience and Regime Shifts in Temperate Forest

Finally, we store the names, institutes and PhD topics to a csv file.

In [5]:
# Create DataFrame and save CSV
df = pd.DataFrame(profiles)
df.to_csv("phd_topics.csv", index=False)

df.head()

,name,research_field,topic
0,Taylor Reed,Chemicals in the Environment / Ecotoxicology,Microplastic-Associated Persistent Organic Pol...
1,Riley Jain,Water Resources and Environment / Aquatic Ecos...,Microbial Community Resilience to Agricultural...
2,Taylor Adams,Ecosystems of the Future / Conservation Biolog...,Resilience and Relocation: Social-Ecological P...
3,Devon Thomas,Ecosystems of the Future / Ecology of Agroecos...,Resilience and Adaptive Capacity: Integrating ...
4,Alex Lee,Chemicals in the Environment / Computational B...,Predicting Persistent Organic Pollutant Bioacc...
